In [2]:
import requests
import json
import re
import psycopg2
import pandas as pd

def execute_sql_code_from_string(input_string):
    sql_blocks = re.findall(r'```sql\s*(.*?)\s*```', input_string, re.DOTALL | re.IGNORECASE)
    
    if not sql_blocks:
        print("No SQL block found.")
        return None
    
    try:
        conn = psycopg2.connect(
            dbname="synthea",
            user="admin",
            password="adminpassword",
            host="localhost",
            port="5432"
        )
        conn.autocommit = True
        cursor = conn.cursor()

        for sql_code in sql_blocks:
            print(f"Executing SQL:\n{sql_code}\n---")
            cursor.execute(sql_code)

            if cursor.description:
                # Get column names
                colnames = [desc[0] for desc in cursor.description]
                # Fetch all data
                rows = cursor.fetchall()
                # Convert to DataFrame
                df = pd.DataFrame(rows, columns=colnames)
                return df

        return None  

    except psycopg2.Error as e:
        print(f"SQL Execution Error: {e}")
        return None

    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()
            
def groq_chat_completion_stream_clean(prompt, model="llama3-8b-8192"):
    GROQ_API_KEY = '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'
    if not GROQ_API_KEY:
        raise ValueError("La clé API Groq n'est pas configurée dans le fichier .env")
    
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {GROQ_API_KEY}"
    }
    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "stream": True  
    }
    response_text = ""
    with requests.post(url, headers=headers, json=data, stream=True) as response:
        if response.status_code != 200:
            raise Exception(f"Erreur API: {response.status_code} - {response.text}")
        
        for chunk in response.iter_lines():
            if chunk:
                decoded_chunk = chunk.decode('utf-8')
                if decoded_chunk.startswith("data:"):
                    try:
                        parsed = json.loads(decoded_chunk[5:].strip())
                        content = parsed.get("choices", [{}])[0].get("delta", {}).get("content")
                        if content:
                            response_text += content
                    except json.JSONDecodeError:
                        continue  

    return response_text.strip()

In [24]:
import json

with open("/home/petriscyril/Desktop/Agent_ETL/prompt/prompt2.txt", "r", encoding="utf-8") as f:
    prompt = f.read()
with open("location2.json", "r") as f:
    json_data = json.load(f)

json_str = json.dumps(json_data, indent=2)

final_prompt = prompt + "\n\n" + json_str

result = groq_chat_completion_stream_clean(final_prompt)

print(result)

Here is the optimized SQL code that transforms data from the source database (A) to the target database (B) according to the provided JSON:

```sql
-- Create a view to perform the transformation
CREATE VIEW transformed_data AS
SELECT 
    -- Use row_number to generate a unique location_id
    ROW_NUMBER() OVER (PARTITION BY city, state, zip) AS location_id
    , NULL AS address_1
    , NULL AS address_2
    , city
    , -- Perform a lookup for state abbreviation
    (SELECT state_abbreviation FROM omop.states_map WHERE state = patients.state) AS state
    , NULL AS county
    , zip
    , zip AS location_source_value
FROM 
    synthea.patients;

-- Create the target table with the transformed data
CREATE TABLE omop.location AS
SELECT 
    location_id
    , address_1
    , address_2
    , city
    , state
    , county
    , zip
    , location_source_value
FROM 
    transformed_data;
```

Note:

* I've created a view `transformed_data` to perform the transformation, which improves modular

prompt1 avec location 8

Here is the generated SQL code:

```sql
CREATE VIEW omop.location_view AS
WITH distinct_locations AS (
    SELECT 
      ROW_NUMBER() OVER (PARTITION BY city, state, zip ORDER BY city, state, zip) AS location_id,
      city,
      s.state_abbreviation AS state, 
      zip,
      zip AS location_source_value
    FROM 
      synthea.patients p
      LEFT JOIN omop.states_map s ON p.state = s.state
  )
SELECT 
  location_id,
  city,
  state,
  zip,
  location_source_value,
  CAST(NULL AS VARCHAR) AS address_1,
  CAST(NULL AS VARCHAR) AS address_2,
  CAST(NULL AS VARCHAR) AS county
FROM 
  distinct_locations;

CREATE TABLE omop.location AS 
SELECT * FROM omop.location_view;
```

This SQL code generates a `VIEW` that performs the desired transformation, and then creates the `omop.location` table based on the view. Note that I've used a CTE (Common Table Expression) to compute the `location_id` and perform the `state` lookup. I've also explicitly cast the `NULL` values to `VARCHAR` type as specified in the JSON.

prompt1 avec location llama8b mark = 8

Here is the optimized SQL code that transforms the data from the source table "synthea.patients" to the target table "omop.location" according to the provided JSON:
```sql
-- Create a view to perform the distinct grouping and transformations
CREATE VIEW location_transform AS
SELECT 
  -- Use ROW_NUMBER() to generate a unique location_id
  ROW_NUMBER() OVER (PARTITION BY p.city, p.state, p.zip) AS location_id,
  p.city,
  -- Perform the lookup on the omop.states_map table
  sm.state_abbreviation AS state,
  p.zip,
  p.zip AS location_source_value,
  -- Set address_1, address_2, and county to NULL
  CAST(NULL AS VARCHAR) AS address_1,
  CAST(NULL AS VARCHAR) AS address_2,
  CAST(NULL AS VARCHAR) AS county
FROM 
  synthea.patients p
  -- Join with the omop.states_map table for lookup
  LEFT JOIN omop.states_map ON p.state = sm.state;

-- Create the target table omop.location from the view
CREATE TABLE omop.location AS
SELECT 
  *
FROM 
  location_transform;
```
Note that I've followed the best practices to make the code efficient, readable, and maintainable. I've used a view to modularize the complex transformation and made sure to limit the selected columns. I've also used a join instead of a subquery for the lookup operation. Additionally, I've applied casting to ensure the correct data type for the address fields.

In [ ]:
import psycopg2
from datetime import datetime
import pytest

def test_table_person_structure(docker_db):
    expected_columns = {
        'person_id': 'integer',
        'gender_concept_id': 'integer',
        'year_of_birth': 'integer',
        'month_of_birth': 'integer',
        'day_of_birth': 'integer',
        'birth_datetime': 'timestamp without time zone', 
        'race_concept_id': 'integer',
        'ethnicity_concept_id': 'integer',
        'location_id': 'integer',
        'provider_id': 'integer',
        'care_site_id': 'integer',
        'person_source_value': 'character varying',  
        'gender_source_value': 'character varying',
        'gender_source_concept_id': 'integer',
        'race_source_value': 'character varying',
        'race_source_concept_id': 'integer',
        'ethnicity_source_value': 'character varying',
        'ethnicity_source_concept_id': 'integer'
    }
    
    id_columns = [
        'person_id',
        'gender_concept_id',
        'race_concept_id',
        'ethnicity_concept_id',
        'location_id',
        'provider_id',
        'care_site_id',
        'gender_source_concept_id',
        'race_source_concept_id',
        'ethnicity_source_concept_id'
    ]
    
    with psycopg2.connect(
        host="localhost",
        database="omop",
        user="admin",
        password="adminpassword",
        port="5432"
    ) as conn:
        with conn.cursor() as cursor:
            cursor.execute("""
                SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = 'person'
                AND table_schema = 'public';  -- replace with your schema if not 'public'
            """)
            
            actual_columns = {row[0]: row[1] for row in cursor.fetchall()}
            
            missing_columns = set(expected_columns.keys()) - set(actual_columns.keys())
            assert not missing_columns, f"Missing columns: {missing_columns}"
            
            type_mismatches = []
            for column, expected_type in expected_columns.items():
                if column in actual_columns and actual_columns[column] != expected_type:
                    type_mismatches.append(
                        f"{column}: expected {expected_type}, got {actual_columns[column]}"
                    )
            
            assert not type_mismatches, "Type mismatches:\n" + "\n".join(type_mismatches)
            
            # Check for NULL values in ID columns
            null_check_issues = []
            for id_column in id_columns:
                if id_column in actual_columns:
                    cursor.execute(f"""
                        SELECT COUNT(*) 
                        FROM person 
                        WHERE {id_column} IS NULL;
                    """)
                    null_count = cursor.fetchone()[0]
                    
                    if null_count > 0:
                        null_check_issues.append(
                            f"{id_column} has {null_count} NULL values"
                        )
            
            assert not null_check_issues, "NULL value issues in ID columns:\n" + "\n".join(null_check_issues)
        

In [26]:
person =  pd.read_csv('/home/petriscyril/Desktop/OMOP/OMOP_dataset/person.csv')
person[0:5]

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,1,8532,2005,7,6,2005-07-06,0,38003563,314,NaN,NaN,0002513e-8009-d8c4-9bf8-bdbb316deae8,F,0,native,0,hispanic,0
1,2,8532,2000,10,18,2000-10-18,8527,38003563,96,NaN,NaN,00035f01-cb9a-d253-eb67-7007a4e19ded,F,0,white,0,hispanic,0
2,3,8532,2001,9,25,2001-09-25,8527,38003563,103,NaN,NaN,000b8952-a1f1-e576-834c-d55c9d7b0941,F,0,white,0,hispanic,0
3,4,8532,1989,10,1,1989-10-01,8527,38003563,632,NaN,NaN,001046c2-98bd-1b63-14c4-ab8f9a7ddfdc,F,0,white,0,hispanic,0
4,5,8532,1992,11,10,1992-11-10,8527,38003563,456,NaN,NaN,0011424c-182d-59ac-5942-056a7f68d983,F,0,white,0,hispanic,0


In [6]:
!ls

docker-compose.yml  initdb.d  location2.json  prompt	      test.py
file.ipynb	    init.sh   location.json   states_map.csv
